In [30]:
import numpy as np
import pandas as pd
import random as random
import matplotlib.pyplot as plt
from scipy import stats 
from datetime import datetime
import wrds
import statsmodels.api as sm
# import statsmodels.api as sm
#db = wrds.Connection(wrds_username = 'hienle') 


In [31]:
# Berkshire Hathaway 
#BRK = db.raw_sql("select date,ret from crsp.dsf where permno in (17778) and date >='1976-01-01' and date<='2019-12-31'")
ff_100=pd.read_csv("F-F_Research_Data_Factors.csv", sep=",")
ff_5_100=pd.read_csv("F-F_Research_Data_5_Factors_2X3.csv", sep=",")
ff_5=ff_5_100/100
ff=ff_100/100
ff_mom=(pd.read_csv("ffm.CSV", sep=","))/100
average_rf=(np.mean(ff['RF'])*12)
ff.head()
#len(BRK)

#BRK

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,1976.01,0.1216,0.0480,0.0858,0.0047
1,1976.02,0.0032,0.0703,0.0578,0.0034
2,1976.03,0.0232,-0.0117,-0.0004,0.0040
3,1976.04,-0.0149,-0.0003,-0.0006,0.0042
4,1976.05,-0.0134,-0.0124,-0.0132,0.0037


In [32]:
#BRK.to_csv("brk_as5_ex3.csv")
BRK = pd.read_csv("brk_as5_ex3.csv").set_index(["date"]).sort_index()[["ret"]]#.dropna()


## PART B

In [33]:
print("Berkshire Hathaway:")
print("Berkshire Hathaway's annualized excess returns mean:",np.mean(BRK)*252-average_rf)
print("Berkshire Hathaway's execss annualized returns' standard deviation:",np.std(BRK)*np.sqrt(252))
print("Berkshire's annualized returns' Sharpe Ratio:",((np.mean(BRK)*252)-average_rf)/(np.std(BRK)*np.sqrt(252)))
print(" ")
print("Market:")
print("The Market's annualized excess returns mean",np.mean(ff['Mkt-RF'])*12)
print("The Market's annualized excess returns' standard deviation",np.std(ff['Mkt-RF'])*np.sqrt(12))
print("The Market's annualized returns' Sharpe Ratio",np.mean(ff['Mkt-RF'])/np.std(ff['Mkt-RF'])*np.sqrt(12))
print(" ")
print("SMB:")
print("The SMB's annualized excess returns mean",np.mean(ff['SMB'])*12)
print("The SMB's annualized excess returns' standard deviation",np.std(ff['SMB'])*np.sqrt(12))
print("The SMB's annualized returns' Sharpe Ratio",np.mean(ff['SMB'])/np.std(ff['SMB'])*np.sqrt(12))
print(" ")
print("HML:")
print("The HML's annualized excess returns mean",np.mean(ff['HML'])*12)
print("The HML's annualized excess returns' standard deviation",np.std(ff['HML'])*np.sqrt(12))
print("The HML's annualized returns' Sharpe Ratio",np.mean(ff['HML'])/np.std(ff['HML'])*np.sqrt(12))
print(" ")
print("MOM:")
print("The MOM's annualized excess returns mean",np.mean((ff_mom)*12))
print("The MOM's annualized excess returns' standard deviation",np.std(ff_mom)*np.sqrt(12))
print("The MOM's annualized returns' Sharpe Ratio",np.mean(ff_mom)/np.std(ff_mom)*np.sqrt(12))

Berkshire Hathaway:
Berkshire Hathaway's annualized excess returns mean: ret    0.182903
dtype: float64
Berkshire Hathaway's execss annualized returns' standard deviation: ret    0.241099
dtype: float64
Berkshire's annualized returns' Sharpe Ratio: ret    0.758621
dtype: float64
 
Market:
The Market's annualized excess returns mean 0.08009149338374288
The Market's annualized excess returns' standard deviation 0.15118876622527921
The Market's annualized returns' Sharpe Ratio 0.5297450027762137
 
SMB:
The SMB's annualized excess returns mean 0.0231833648393195
The SMB's annualized excess returns' standard deviation 0.10267887836666106
The SMB's annualized returns' Sharpe Ratio 0.22578513914549084
 
HML:
The HML's annualized excess returns mean 0.02913799621928169
The HML's annualized excess returns' standard deviation 0.09994134805358647
The HML's annualized returns' Sharpe Ratio 0.29155096250711465
 
MOM:
The MOM's annualized excess returns mean Mom    0.074268
dtype: float64
The MOM's 

## PART C

In [34]:
BRK.index = pd.DatetimeIndex(BRK.index)
BRK_monthly = BRK.resample("1m").mean()*21 #take the first or last return of each month, or choose another function
BRK_monthly.index = BRK_monthly.index.to_period("m")



In [35]:
np.mean(BRK_monthly)

ret    0.020351
dtype: float64

In [36]:

ff = ff.iloc[9:528]
len(ff)

519

In [37]:
ff_mom = ff_mom.iloc[9:528]
ff_mom.index = ff.index
len(ff_mom)

519

In [38]:
ff_5.index = ff.index
ff_3m=pd.concat([ff, ff_mom], axis=1)
ff_5m=pd.concat([ff_5, ff_mom], axis=1)
ff_5m.columns

Index(['Unnamed: 0', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom'], dtype='object')

In [39]:
def ols(covariates,flag,regressand='BRK'):
    y=R_e_brk['BRK']
    X=flag[covariates]
    X=sm.add_constant(X)
    reg=sm.OLS(y,X).fit()
    print(reg.summary())
    print("The corresponding Information Ratio is",np.divide(reg.params[0],np.sqrt(reg.mse_resid))*np.sqrt(12))
    #elif flag==ff_5m:
       # X=ff_3m[covariates]
        #X=sm.add_constant(X)
        #reg=sm.OLS(y,X).fit()
        #print(reg.summary())
        #print("The corresponding Information Ratio is",np.divide(reg.params[0],np.sqrt(reg.mse_resid))*np.sqrt(12))


In [40]:
R_e_brk=pd.DataFrame((np.array(BRK_monthly['ret']) - np.array(ff['RF'])),columns=['BRK'])
R_e_brk.index = ff.index
ols('Mkt-RF',ff_3m)


OLS Regression Results                            
Dep. Variable:                    BRK   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     98.19
Date:                Wed, 18 Mar 2020   Prob (F-statistic):           2.62e-21
Time:                        22:23:26   Log-Likelihood:                 673.82
No. Observations:                 519   AIC:                            -1344.
Df Residuals:                     517   BIC:                            -1335.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0124      0.003      4.234      0.000       0.007       0.018
M

In [41]:
ols(['Mkt-RF','SMB','HML'],ff_3m)

OLS Regression Results                            
Dep. Variable:                    BRK   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     46.79
Date:                Wed, 18 Mar 2020   Prob (F-statistic):           9.38e-27
Time:                        22:23:26   Log-Likelihood:                 691.24
No. Observations:                 519   AIC:                            -1374.
Df Residuals:                     515   BIC:                            -1357.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0110      0.003      3.834      0.000       0.005       0.017
M

In [42]:
ols(['Mkt-RF','SMB','HML','Mom'],ff_3m)

OLS Regression Results                            
Dep. Variable:                    BRK   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     35.25
Date:                Wed, 18 Mar 2020   Prob (F-statistic):           4.98e-26
Time:                        22:23:27   Log-Likelihood:                 691.60
No. Observations:                 519   AIC:                            -1373.
Df Residuals:                     514   BIC:                            -1352.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0105      0.003      3.602      0.000       0.005       0.016
M

In [43]:
ols(['Mkt-RF','SMB','HML','RMW', 'CMA','Mom'],ff_5m)

OLS Regression Results                            
Dep. Variable:                    BRK   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     24.42
Date:                Wed, 18 Mar 2020   Prob (F-statistic):           1.78e-25
Time:                        22:23:27   Log-Likelihood:                 694.00
No. Observations:                 519   AIC:                            -1374.
Df Residuals:                     512   BIC:                            -1344.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0093      0.003      3.098      0.002       0.003       0.015
M

In [23]:
##PART H

In [53]:
ff3_95 = ff_3m.iloc[:231]
ff5_95 = ff_5m.iloc[:231]